# **Redes Neuronales Artificiales**: Trabajo Práctico 1

#### Integrantes:

- Maximiliano Dacko, LU  284/21
- María Marino, LU 450/21
- Giovanni Marraffini, LU 292/21

## Clase `MLP`: Multi Level Perceptron

En primer lugar, se crea una clase para facilitar las acciones más utilizadas por la red neuronal. Esta consta de un constructor donde se definen la cantidad de neuronas de cada capa y la función de activación. Asimismo se crean las matrices pseudoaleatorias con una distribución normal.

Además, se utilizan como métodos públicos una función de activación de la red, una función de corrección de los pesos que utiliza el algortimo de *backpropagation* del error y una función para obtener los valores de salida dados nuevos datos de entrada.

Se decidió utilizar la misma función de activación para todas las capas. 

In [22]:
import numpy as np

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

class MLP:
    def __init__(self, sizes: list[int], batchSize: int, f: str = 'tanh'):
        self.S = sizes
        self.L = len(self.S)
        self.N = batchSize
        self.Y = [np.empty(shape=(self.N, self.S[i - 1] + 1)) for i in range(self.L)]
        self.W = [None] + [np.random.normal(0, 0.5, (self.S[i - 1] + 1, self.S[i])) for i in range(1, self.L)]
        self._f = f

    
    def _add_bias(self, V):
        bias = np.ones((len(V),1))
        return np.hstack([V, bias])

    def _sub_bias(self, V):
        return V[:, :-1]
    
    @property
    def f(self):
        return self._f

    @f.setter
    def f(self, fun):
        if fun != 'tanh' and fun != 'sigmoid': 
            raise Exception('Activation function should be  \'tanh\' or \'sigmoid\'')
    
    def activation(self, Xh):
        self.Y = [np.empty(shape=(self.N, self.S[i - 1] + 1)) for i in range(self.L)]
        Y_b = Xh
        for k in range(1, self.L):
            self.Y[k - 1] = self._add_bias(Y_b)
            if self.f == 'sigmoid':
                Y_b = sigmoid(self.Y[k - 1] @ self.W[k])
            else:
                Y_b = np.tanh(self.Y[k - 1] @ self.W[k])
        self.Y[self.L - 1] = Y_b
        return self.Y
    
    def correction(self, Yh, Zh):
        D = [np.empty(shape=(self.N, self.S[i - 1] + 1)) for i in range(self.L)]
        deltaW = [None] + [np.empty(shape=(self.S[i - 1] + 1, self.S[i])) for i in range(1, self.L)]
        E = Zh - Yh[self.L - 1]
        if self.f == 'sigmoid':
            dY = Yh[self.L - 1] * (1 - Yh[self.L - 1]) # Derivative of sigmoid
        else:
            dY = 1 - np.square(Yh[self.L - 1]) # Derivative of tanh
        D[self.L - 1] = E * dY
        for k in reversed(range(1, self.L)):
            deltaW[k] = Yh[k - 1].T @ D[k]
            E = D[k] @ self.W[k].T
            if self.f == 'sigmoid':
                dY = Yh[k - 1] * (1 - Yh[k - 1]) # Derivative of sigmoid
            else:
                dY = 1 - np.square(Yh[k - 1]) # Derivative of tanh
            D[k - 1] = self._sub_bias(E * dY)
        return deltaW

    def predict(self, x):
      return self.activation(x)[-1]

## **Ejercicio 1**

Se subieron los datos a un repositorio público para facilitar su importación.

Por el enunciado, sabemos que el conjunto de datos consta de las siguientes columnas:
1. *Diagnóstico, dónde 1 = maligno y 0 = benigno*. (**Nuestra variable objetivo**).
2. *Radio: media de la distancia desde el centro a los puntos deperímetro*
3. *Textura: desviación estándar de los valores en escala de gris*
4. *Perímetro*
5. *Área*
6. *Suavidad: variaciones locales en la longitud del radio*
7. *Compacidad: perímetro^2 / área - 1*
8. *Concavidad: severidad de las porciones cóncavas del contorno*
9. *Puntos cóncavos: proporción de porciones cóncavas del contorno*
10. *Simetría*
11. *Dimensión fractal*

In [2]:
data = np.loadtxt('https://raw.githubusercontent.com/marinomaria/UBA-redes_neuronales/main/dataset/tp1-ej1.txt', delimiter = ',')

datasetSize = data.shape[0]

inputSize = 10
outputSize = 1

Se decide separar el 10% de los datos para luego evaluar con estos al final del entrenamiento y obtener un estimado del error real que podría llegar a cometer el modelo.

In [3]:
np.random.shuffle(data)

evalSetSize = int(datasetSize * 0.1)
trainSet = data[: - evalSetSize, :]
evalSet = data[- evalSetSize:, :]

trainSet.shape, evalSet.shape

((369, 11), (41, 11))

### Estandarización

Para cada variable miramos su rango y promedio, y luego procedemos a estandarizar todo el dataset de entrenamiento **sin** la variable objetivo:

In [4]:
for j in range(trainSet.shape[1]):
  print(f'Columna {j}: \n Rango: {trainSet[:, j].min()} - {trainSet[:, j].max()} \n Promedio: {trainSet[:, j].mean()}')

Columna 0: 
 Rango: 0.0 - 1.0 
 Promedio: 0.4796747967479675
Columna 1: 
 Rango: 6.981 - 28.11 
 Promedio: 18.55281842818428
Columna 2: 
 Rango: 10.034 - 33.81 
 Promedio: 21.26975609756098
Columna 3: 
 Rango: 43.79 - 188.5 
 Promedio: 113.85478861788619
Columna 4: 
 Rango: 143.5 - 2501.0 
 Promedio: 844.3650433604336
Columna 5: 
 Rango: 0.053 - 4.163 
 Promedio: 1.790352303523035
Columna 6: 
 Rango: 0.387 - 3.345 
 Promedio: 1.8854715447154473
Columna 7: 
 Rango: 0.427 - 7.283 
 Promedio: 3.6615067750677506
Columna 8: 
 Rango: 0.201 - 9.013 
 Promedio: 4.548010840108401
Columna 9: 
 Rango: 0.106 - 1.304 
 Promedio: 0.728018970189702
Columna 10: 
 Rango: 0.094 - 2.974 
 Promedio: 1.5555880758807588


In [5]:
z = trainSet[:, 0].reshape(trainSet.shape[0], 1)
x = trainSet[:, 1:trainSet.shape[1]]

z.shape, x.shape

((369, 1), (369, 10))

La estandarización utilizada es:

$z = \dfrac{x - \mu}{\sigma}$

donde $x$ es el valor original, $\mu$ y $\sigma$ son respectivamente la media y la desviación estándar de la variable.

In [6]:
x = (x - x.mean(0)) / x.std(0)

### Entrenamiento

Definimos el *batch size* (tamaño del lote) como un 10% del total de nuestro dataset. Teniendo en cuenta que ya quitamos un 10% para validación, nos quedan 9 lotes para entrenamiento.

Asimismo inicializamos una instancia de la clase `MLP`, definiendo aquí parte de su arquitectura. En este caso, nuestra red neuronal tendrá una única capa oculta cuyo tamaño será el doble del tamaño de entrada, y todas las capas utilizarán la función sigmoide como función de activación.

In [7]:
bSize = int(datasetSize * 0.1)
M = MLP([inputSize, inputSize * 2, outputSize], bSize, 'sigmoid')

Entrenamos el modelo de a lotes. 

Cada ciclo del `while loop` exterior representa una época (*epoch*) y en él permutamos el orden de los datos para intentar evitar estancarnos en un mínimo local de nuestra función de error.

En cada iteración del `for loop` ingresamos un lote al modelo, activando la red y luego corrigiendo sus pesos con los métodos ya mencionados de la clase `MLP`.

Para calcular el error, sumamos el cuadrado de la diferencia entre el valor esperado (variable objetivo) y el valor predicho por el modelo, y luego dividimos por la cantidad de términos de la suma (error cuadrático medio).

Por último, imprimimos para algunas épocas el error estimado para poder observar su evolución.

In [8]:
minError = 0.001
maxEpoch = 40000
e = 1.0
eta = 0.001
epoch = 0
p = x.shape[0]

while e > minError and epoch < maxEpoch:
    stochastic = np.random.permutation(p)
    e = 0.0
    for i in range(0, p, bSize):
        batch = stochastic[i : i + bSize]
        xi = x[batch].reshape(bSize, 10)
        zi = z[batch].reshape(bSize, 1)
        yi = M.activation(xi)
        e += np.sum(np.square(zi - yi[-1]))
        dW = M.correction(yi, zi)
        for j in range(1, M.L):   
          M.W[j] += eta * dW[j]
    e /= (p / bSize)
    if (epoch % 10000 == 0 or epoch == 39999):
      print(f'Epoch {epoch}, error: {e}')
    epoch += 1 

epoch: 0 error: 11.309736414143662
epoch: 10000 error: 2.835541017156794
epoch: 20000 error: 0.5508116212977473
epoch: 30000 error: 0.22391331909331255
epoch: 39999 error: 0.12583386340479386


### Validación

Para validar el desempeño del modelo lo utilizamos para predecir el resultado del 10% del dataset que **no fue utilizado en su entrenamiento**.

Primero estandarizamos este dataset de validación como hicimos con los datos de entrenamiento, y luego comparamos los resultados obtenidos con los esperados.

In [13]:
zEval = evalSet[:, 0:1].reshape(evalSet.shape[0], 1)
xEval = evalSet[:, 1:11]
xEval = (xEval - xEval.mean(0)) / xEval.std(0)

pred = np.zeros_like(zEval)

for i in range(len(evalSet)):
  pred[i] = M.predict(xEval[i].reshape(1, 10))

print(f'El modelo acertó en {np.sum(zEval == np.round(pred))} de {len(evalSet)} instancias ({100 * np.round(np.sum(zEval == np.round(pred)) / len(evalSet), 2)} %).\n\n')

for i in range(len(zEval)):
  print(f'Esperado: {zEval[i]}  Predicho: {pred[i]}   Redondeado: {np.round(pred[i])}')

El modelo acertó en 40 de 41 instancias (98.0 %).


Esperado: [1.]  Predicho: [0.86864599]   Redondeado: [1.]
Esperado: [0.]  Predicho: [0.05837362]   Redondeado: [0.]
Esperado: [0.]  Predicho: [0.0023144]   Redondeado: [0.]
Esperado: [0.]  Predicho: [0.0038575]   Redondeado: [0.]
Esperado: [0.]  Predicho: [0.00422025]   Redondeado: [0.]
Esperado: [0.]  Predicho: [0.46358652]   Redondeado: [0.]
Esperado: [1.]  Predicho: [0.98712332]   Redondeado: [1.]
Esperado: [1.]  Predicho: [0.84075463]   Redondeado: [1.]
Esperado: [1.]  Predicho: [0.98283279]   Redondeado: [1.]
Esperado: [0.]  Predicho: [0.0390161]   Redondeado: [0.]
Esperado: [0.]  Predicho: [0.01116623]   Redondeado: [0.]
Esperado: [0.]  Predicho: [0.08248375]   Redondeado: [0.]
Esperado: [1.]  Predicho: [0.74606755]   Redondeado: [1.]
Esperado: [0.]  Predicho: [0.9236483]   Redondeado: [1.]
Esperado: [1.]  Predicho: [0.98304546]   Redondeado: [1.]
Esperado: [1.]  Predicho: [0.99563944]   Redondeado: [1.]
Esperado: [0.]  Predicho

#### Validación cruzada con *K-folds*

Por último, para tener un mayor entendimiento de la performance del modelo en el "caso real" utilizamos el método *K-folds*.

La siguiente celda de código demora más en correr ya que entrena múltiples modelos con la misma arquitectura que el original pero variando los conjuntos de datos de entrenamiento y validación.

In [18]:
s = np.random.permutation(datasetSize)

pred = np.empty(shape=(datasetSize, outputSize))

print(f'Cantidad de folds: {int(datasetSize / bSize)}. Tamaño de cada fold: {bSize} instancias.')

for i in range(0, datasetSize, bSize):
    fold = s[i : i + 82]
    evalSet = data[fold]
    trainSet = np.delete(data, fold, 0)

    x = trainSet[:, 1:11]
    z = trainSet[:, 0]
    
    x = (x - x.mean(0)) / x.std(0)

    M = MLP([inputSize, inputSize * 2, outputSize], bSize, 'sigmoid')

    e = 1.0
    epoch = 0
    p = len(trainSet)

    while e > minError and epoch < maxEpoch:
        stochastic = np.random.permutation(p)
        e = 0.0
        for j in range(0, p, bSize):
            batch = stochastic[j : j + bSize]
            xj = x[batch].reshape(bSize, M.S[0])
            zj = z[batch].reshape(bSize, M.S[-1])
            y = M.activation(xj)
            e += np.sum(np.square(zj - y[-1]))
            dW = M.correction(y, zj)
            for k in range(1, M.L):
                M.W[k] += eta * dW[k]
        e /= (p / bSize)
        epoch += 1 

    xEval = evalSet[:, 1:11]
    zEval = evalSet[:, 0]

    xEval = (xEval - xEval.mean(0)) / xEval.std(0)

    pred[fold] = M.predict(xEval)
    print(f'Modelo entrenado sin el fold {int(i / bSize)}. Error en la última época: {e}')



Cantidad de folds: 10. Tamaño de cada fold: 41 instancias.
Modelo entrenado sin el fold 0. Error en la última época: 0.24977040908568715
Modelo entrenado sin el fold 1. Error en la última época: 0.15338632601200874
Modelo entrenado sin el fold 2. Error en la última época: 0.17776726263485912
Modelo entrenado sin el fold 3. Error en la última época: 0.17810413681260756
Modelo entrenado sin el fold 4. Error en la última época: 0.14886187134989365
Modelo entrenado sin el fold 5. Error en la última época: 1.677910403886872
Modelo entrenado sin el fold 6. Error en la última época: 0.1587617722050359
Modelo entrenado sin el fold 7. Error en la última época: 0.16037834493560219
Modelo entrenado sin el fold 8. Error en la última época: 0.13559293903745667
Modelo entrenado sin el fold 9. Error en la última época: 0.15620125617833264


Los *K-folds* se organizan de forma tal que para cada instancia se obtiene un resultado con alguno de los $K$ modelos entrenados. Guardamos esos resultados en el arreglo `pred` y los comparamos con los valores esperados para construir las métricas de *Recall* y Precisión.

$\text{Recall} = \dfrac{\text{Verdaderos Positivos}}{\text{Verdaderos Positivos} + \text{Falsos Negativos}}$

$\text{Precisión} = \dfrac{\text{Verdaderos Positivos}}{\text{Verdaderos Positivos} + \text{Falsos Positivos}}$


In [21]:
expected = data[:, 0].reshape(datasetSize, outputSize)

tp = fn = tn = fp = 0

for i in range(len(pred)):
    if expected[i] == 1:
        if np.round(pred[i]) == 1:
            tp += 1
        else:
            fn += 1
    else:
        if np.round(pred[i]) == 1:
            fp += 1
        else:
            tn += 1

print('Recall:', tp / (tp + fn))
print('Precisión:', tp / (tp + fp))

print(f'\n\nSe cometieron {fp + fn} errores en {datasetSize} diagnósticos.')
print(f'Diagnosticamos como benignos a {fn} tumores malignos.')
print(f'Diagnosticamos como malignos a {fp} tumores benignos.')

Recall: 0.8871794871794871
Precisión: 0.9105263157894737


Se cometieron 39 errores en 410 diagnósticos.
Diagnosticamos como benignos a 22 tumores malignos.
Diagnosticamos como malignos a 17 tumores benignos.


En este caso la medida principal que queremos optimizar es el *recall*, ya que al tratarse de un problema de diagnóstico de cáncer de mama es preferible sobreestimar (dar un resultado positivo para tumores benignos) que subestimar, por el impacto que puede tener en las posibilidades de tratamiento y en la calidad de vida de un paciente la detección tardía de un tumor maligno.

Como alternativa para mejorar nuestro modelo podríamos agregar un *threshold* $\epsilon$, y sólo diagnosticar como benignos aquellos casos donde el modelo devuelva resultados menores a $0,5 + \epsilon$.